In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [2]:
# MultiHeadAttention layer
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out


In [3]:
# ViTBlock
class ViTBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout):
        super(ViTBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query):
        attention = self.attention(value, key, query)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [4]:
# Vision Transformer (ViT) model
class ViT(nn.Module):
    def __init__(
        self,
        in_channels,
        embed_size,
        num_classes,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        image_size,
        patch_size,
    ):
        super(ViT, self).__init__()
        self.embed_size = embed_size
        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2

        self.patch_embedding = nn.Linear(patch_size * patch_size * in_channels, embed_size)
        self.position_embedding = nn.Parameter(torch.rand(self.num_patches + 1, embed_size))

        self.cls_token = nn.Parameter(torch.rand(1, 1, embed_size))
        self.dropout = nn.Dropout(dropout)

        self.layers = nn.ModuleList(
            [
                ViTBlock(
                    embed_size,
                    heads,
                    forward_expansion=forward_expansion,
                    dropout=dropout,
                )
                for _ in range(num_layers)
            ]
        )

        self.fc = nn.Linear(embed_size, num_classes)

    def forward(self, x):
        N, _, _, _ = x.shape
        x = x.view(N, self.num_patches, -1)
        x = self.patch_embedding(x)

        cls_tokens = self.cls_token.expand(N, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.position_embedding

        x = self.dropout(x)

        for layer in self.layers:
            x = layer(x, x, x)

        x = self.fc(x[:, 0, :])

        return x

In [5]:
# Data augmentation and loaders for CIFAR-10
transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

train_dataset = datasets.CIFAR10(root="dataset/", train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root="dataset/", train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:03<00:00, 49258973.33it/s]


Extracting dataset/cifar-10-python.tar.gz to dataset/
Files already downloaded and verified


In [6]:
model = ViT(
    in_channels=3,
    embed_size=256,
    num_classes=10,
    num_layers=6,
    heads=8,
    forward_expansion=4,
    dropout=0.1,
    image_size=32,
    patch_size=8,
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(data)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}")

    model.eval()
    num_correct = 0
    num_samples = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data = data.to(device)
            targets = targets.to(device)

            outputs = model(data)

            _, predictions = outputs.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

    print(f"Accuracy: {float(num_correct) / float(num_samples) * 100:.2f}%")

Epoch [1/20], Step [0/782], Loss: 2.4632
Epoch [1/20], Step [100/782], Loss: 2.2889
Epoch [1/20], Step [200/782], Loss: 2.1539
Epoch [1/20], Step [300/782], Loss: 1.9662
Epoch [1/20], Step [400/782], Loss: 1.8807
Epoch [1/20], Step [500/782], Loss: 2.0061
Epoch [1/20], Step [600/782], Loss: 1.8846
Epoch [1/20], Step [700/782], Loss: 1.7877
Accuracy: 32.82%
Epoch [2/20], Step [0/782], Loss: 1.6988
Epoch [2/20], Step [100/782], Loss: 1.7182
Epoch [2/20], Step [200/782], Loss: 1.7475
Epoch [2/20], Step [300/782], Loss: 1.7043
Epoch [2/20], Step [400/782], Loss: 1.7215
Epoch [2/20], Step [500/782], Loss: 1.7745
Epoch [2/20], Step [600/782], Loss: 1.6220
Epoch [2/20], Step [700/782], Loss: 1.6668
Accuracy: 39.13%
Epoch [3/20], Step [0/782], Loss: 1.6578
Epoch [3/20], Step [100/782], Loss: 1.5914
Epoch [3/20], Step [200/782], Loss: 1.5709
Epoch [3/20], Step [300/782], Loss: 1.8378
Epoch [3/20], Step [400/782], Loss: 1.6573
Epoch [3/20], Step [500/782], Loss: 1.8486
Epoch [3/20], Step [600/78

In [7]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_trainable_params = count_trainable_parameters(model)
print(f"Number of trainable parameters in ViT model: {num_trainable_params}")


Number of trainable parameters in ViT model: 3629322


In [11]:
# Test the ViT model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, targets in test_loader:
        data = data.to(device)
        targets = targets.to(device)

        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f'Accuracy of the ViT model on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the ViT model on the 10000 test images: 60.92%
